In [1]:
# GPUの種類を確認する
!nvidia-smi

Thu Apr  9 12:30:46 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# 京都関連文書対訳コーパス(KFTT)をダウンロードする。
!wget http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz
# tar.gzファイルを解凍する。
!tar -zxvf /content/kftt-data-1.0.tar.gz
# Googleドライブに分かち書き済みの対訳データをコピーする。
!cp -r kftt-data-1.0/data/tok/ 'drive/My Drive/'

--2020-04-09 12:30:48--  http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz
Resolving www.phontron.com (www.phontron.com)... 208.113.196.149
Connecting to www.phontron.com (www.phontron.com)|208.113.196.149|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99246893 (95M) [application/gzip]
Saving to: ‘kftt-data-1.0.tar.gz.2’

kftt-data-1.0.tar.g 100%[===================>]  94.65M  37.1MB/s    in 2.5s    

2020-04-09 12:30:51 (37.1 MB/s) - ‘kftt-data-1.0.tar.gz.2’ saved [99246893/99246893]

kftt-data-1.0/
kftt-data-1.0/data/
kftt-data-1.0/data/orig/
kftt-data-1.0/data/orig/kyoto-tune.en
kftt-data-1.0/data/orig/kyoto-dev.ja
kftt-data-1.0/data/orig/kyoto-dev.en
kftt-data-1.0/data/orig/kyoto-train.en
kftt-data-1.0/data/orig/kyoto-tune.ja
kftt-data-1.0/data/orig/kyoto-train.ja
kftt-data-1.0/data/orig/kyoto-test.ja
kftt-data-1.0/data/orig/kyoto-test.en
kftt-data-1.0/data/tok/
kftt-data-1.0/data/tok/kyoto-tune.en
kftt-data-1.0/data/tok/kyoto-dev.ja
kftt-data-1.0

In [3]:
## ディレクトリの中身を確認する
!ls '/content/drive/My Drive'tok

ls: cannot access '/content/drive/My Drivetok': No such file or directory


In [4]:
# カレントディレクトリを変更する。
%cd drive/My Drive
# フォルダを作成する。
!mkdir data
!mkdir save

/content/drive/My Drive
mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘save’: File exists


In [0]:
def limiter(data_dir, src_data, tgt_data, n=50):
  '''limiter
  コーパスの最大文長を制限する。
  ----------------------------------------
  引数
  data_dir : コーパスのディレクトリ
  src_data : 原言語のデータ
  tgt_data : 目的言語のデータ
  n : 最大文長
  ----------------------------------------
  '''
  # 書き込み用のデータファイルを作成する。
  with open(f'{data_dir}/limit-{src_data}','w', encoding='utf8') as wsrc:
     with open(f'{data_dir}/limit-{tgt_data}','w', encoding='utf8') as wtgt:
       # 元のデータファイルを読み込む
       with open(f'{data_dir}/{src_data}', encoding='utf8') as rsrc:
         with open(f'{data_dir}/{tgt_data}', encoding='utf8') as rtgt:
           # それぞれ一行ずつ読み込みリストに格納する。
           src_lines = rsrc.read().strip().split('\n')
           tgt_lines = rtgt.read().strip().split('\n')
           for src, tgt in zip(src_lines, tgt_lines):
             # 最大文長がn以下のもののみ書き込む。
             if len(src.split()) <= n and len(tgt.split()) <= n:
               wsrc.write(src + '\n')
               wtgt.write(tgt + '\n')

In [0]:
## 文長制限するファイルの一覧
files = ['train','dev', 'test']
for f in files:
  # 最大文長10とする
  limiter('tok', f'kyoto-{f}.en', f'kyoto-{f}.ja' ,n=50)

In [7]:
# ディレクトリの中身を確認する。
!ls tok

fif-limit-kyoto-output.ja  kyoto-train.cln.ja  limit-kyoto-dev.ja
kyoto-dev.en		   kyoto-train.en      limit-kyoto-test.en
kyoto-dev.ja		   kyoto-train.ja      limit-kyoto-test.ja
kyoto-test.en		   kyoto-tune.en       limit-kyoto-train.en
kyoto-test.ja		   kyoto-tune.ja       limit-kyoto-train.ja
kyoto-train.cln.en	   limit-kyoto-dev.en


In [8]:
# 文長制限した時のサイズを確認する。
!wc -l tok/limit-kyoto-train.en
!wc -l tok/limit-kyoto-train.ja

377471 tok/limit-kyoto-train.en
377471 tok/limit-kyoto-train.ja


In [9]:
# OpenNMT-pyをインストールする。
!pip install OpenNMT-py

In [10]:
# データの前処理をする。
!onmt_preprocess -train_src tok/limit-kyoto-train.en \
                 -train_tgt tok/limit-kyoto-train.ja \
                 -valid_src tok/limit-kyoto-dev.en \
                 -valid_tgt tok/limit-kyoto-dev.ja \
                 -save_data data/demo

[2020-04-09 12:31:18,853 INFO] Extracting features...
[2020-04-09 12:31:18,860 INFO]  * number of source features: 0.
[2020-04-09 12:31:18,860 INFO]  * number of target features: 0.
[2020-04-09 12:31:18,860 INFO] Building `Fields` object...
[2020-04-09 12:31:18,860 INFO] Building & saving training data...
[2020-04-09 12:31:18,861 WARNING] Shards for corpus train already exist, won't be overwritten, pass the `-overwrite` option if you want to.
[2020-04-09 12:31:18,861 INFO] Building & saving validation data...
[2020-04-09 12:31:18,862 WARNING] Shards for corpus valid already exist, won't be overwritten, pass the `-overwrite` option if you want to.


In [11]:
# データフォルダを確認する。
!ls data

demo.train.0.pt  demo.valid.0.pt  demo.vocab.pt


In [12]:
# 学習
!onmt_train -data data/demo \
            -save_model save/ \
            -layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8  \
            -encoder_type transformer -decoder_type transformer -position_encoding \
            -train_steps 2000  -max_generator_batches 2 -dropout 0.1 \
            -batch_size 1024 -batch_type tokens -normalization tokens  -accum_count 2 \
            -optim adam -adam_beta2 0.998 -decay_method noam -warmup_steps 4000 -learning_rate 2 \
            -max_grad_norm 0 -param_init 0  -param_init_glorot \
            -label_smoothing 0.1 -valid_steps 2000 -save_checkpoint_steps 2000 \
            -world_size 1 -gpu_ranks 0

[2020-04-09 12:31:22,991 INFO]  * src vocab size = 50002
[2020-04-09 12:31:22,991 INFO]  * tgt vocab size = 50004
[2020-04-09 12:31:22,991 INFO] Building model...
[2020-04-09 12:31:28,802 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)
          (final_linear): Linear(in_features=512, out_feat

In [14]:
# 翻訳
!onmt_translate -model save/_step_2000.pt \
                -src tok/limit-kyoto-test.en \
                -output tok/limit-kyoto-output.ja \
                -replace_unk -verbose \
                -gpu 0

[2020-04-09 13:18:04,808 INFO] Translating shard 0.

SENT 1: ['Infobox', 'Buddhist']
PRED 1: Buddhist
PRED SCORE: -3.3470

SENT 2: ['Dogen', 'was', 'a', 'Zen', 'monk', 'in', 'the', 'early', 'Kamakura', 'period', '.']
PRED 2: 鎌倉 時代 ( 鎌倉 時代 ) に は 鎌倉 時代 に あ る 。
PRED SCORE: -15.9458

SENT 3: ['The', 'founder', 'of', 'Soto', 'Zen']
PRED 3: 概要
PRED SCORE: -2.9657

SENT 4: ['Later', 'in', 'his', 'life', 'he', 'also', 'went', 'by', 'the', 'name', 'Kigen', '.']
PRED 4: これ に よ る 。
PRED SCORE: -7.7179

SENT 5: ['Within', 'the', 'sect', 'he', 'is', 'referred', 'to', 'by', 'the', 'honorary', 'title', 'Koso', '.']
PRED 5: 真言 宗 。
PRED SCORE: -7.4975

SENT 6: ['Posthumously', 'named', 'Bussho', 'Dento', 'Kokushi', ',', 'or', 'Joyo-Daishi', '.']
PRED 6: .
PRED SCORE: -5.6736

SENT 7: ['He', 'is', 'generally', 'called', 'Dogen', 'Zenji', '.']
PRED 7: これ は He 。
PRED SCORE: -8.3799

SENT 8: ['He', 'is', 'reputed', 'to', 'have', 'been', 'the', 'one', 'that', 'spread', 'the', 'practices', 'of', 'tooth', 'br